In [1]:
import pandas as pd
import os
import numpy as np
from numpy import array
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame
import pickle
import time
from tensorflow import keras
from os.path import  join

## 最後要交的

In [12]:
#Test Classification is the prototype of the function that each team must develop to classify new data
#This function must handle all the operation to: read in a streaming order the input file, make the earlier possibile classification, return the required data
#Input: 
# - Folder Name: The name of the folder where the experiment file is stored
# - Experiment: The experiment name that must be read 
#Output:
# - Predicted Label: the label predicted by the classifier
# - Time for classification: how much time of the input data was required to perform the classification task
# - Ranking: The Features ranked according to the team solution

def TestClassification(FolderName,Experiment):
    start = time.time()
    feature_name=["CpuTemperature_3","DurationPickToPick_6","DurationRobotFromFeederToTestBench_6",                                 "DurationRobotFromTestBenchToFeeder_6","DurationTestBenchClosed_6",
                                     "EPOSPosition_6","FeederAction2_0","FeederAction4_0","FeederBackgroundIlluminationIntensity_6","FuseCycleDuration_6",
                                     "FuseHeatSlope_0","FuseHeatSlopeNOK_6","FuseHeatSlopeOK_6","FuseIntoFeeder_0",
                                     "FusePicked_6","FuseTestResult_6","Humidity_0","IntensityTotalImage_0",
                                     "IntensityTotalThermoImage_6","LightBarrierActiveTaskDuration1_6","NumberEmptyFeeder_0",
                                     "NumberFuseDetected_6","NumberFuseEstimated_6","Pressure_6","ProcessCpuLoadNormalized_3",
                                     "ProcessMemoryConsumption_3","SharpnessImage_6","SmartMotorPositionError_6",
                                     "SmartMotorSpeed_6","Temperature_0","TemperatureThermoCam_6","TotalCpuLoadNormalized_3",
                                     "TotalMemoryConsumption_3","Vacuum_6","VacuumFusePicked_6","VacuumValveClosed_6"]
    path_name=join(FolderName,Experiment)
    test_data = Data_adjust(path_name) #input test and clean
    test_feature_dict=Test_feature(test_data,70) #36 feat,timestep=70
    fail_dict,sensor_list=Test_detect(feature_name,test_feature_dict) # detect experinment is fault or not
    if fail_dict != "None":
        Label=Label_classfication(fail_dict)
    else:
        Label=0    
    #Output example to run the script
    end = time.time()
    Time = end - start
    Ranking = ['Humidity','TotalMemoryConsumption','ProcessMemoryConsumption','Pressure','SmartMotorPositionError',
           'VacuumValveClosed','NumberFuseEstimated','Vacuum','IntensityTotalImage','NumberEmptyFeeder',
           'Temperature','TemperatureThermoCam''IntensityTotalThermoImage','NumberFuseDetected',
           'FeederBackgroundIlluminationIntensity','SharpnessImage','FeederAction2','DurationPickToPick',
           'FuseIntoFeeder','TotalCpuLoadNormalized','DurationRobotFromFeederToTestBench',
           'DurationRobotFromTestBenchToFeeder','FusePicked','FuseCycleDuration','EPOSPosition',
           'ProcessCpuLoadNormalized','FuseHeatSlope','LightBarrierPassiveTaskDuration1','FuseHeatSlopeNOK',
           'DurationTestBenchClosed','VacuumFusePicked','CpuTemperature','FuseTestResult','FeederAction4',
           'FuseHeatSlopeOK','SmartMotorSpeed','LightBarrierActiveTaskDuration1','ValidFrameOptrisPIIRCamera',
           'EPOSVelocity','ValidFrame','EPOSCurrent','FeederAction3','FeederAction1','FuseOutsideOperationalSpace',
           'ErrorFrame','LightBarrieActiveTaskDuration2','LightBarrierActiveTaskDuration1b','LightBarrierPassiveTaskDuration1b',
          'LightBarrierPassiveTaskDuration2','LightBarrierTaskDuration']

    return Label, Time, Ranking

In [3]:
def Select_feature(df): #36 features
    df=pd.DataFrame(data=df,columns=["CpuTemperature_3","DurationPickToPick_6","DurationRobotFromFeederToTestBench_6",
                                     "DurationRobotFromTestBenchToFeeder_6","DurationTestBenchClosed_6",
                                     "EPOSPosition_6","FeederAction2_0","FeederAction4_0","FeederBackgroundIlluminationIntensity_6","FuseCycleDuration_6",
                                     "FuseHeatSlope_0","FuseHeatSlopeNOK_6","FuseHeatSlopeOK_6","FuseIntoFeeder_0",
                                     "FusePicked_6","FuseTestResult_6","Humidity_0","IntensityTotalImage_0",
                                     "IntensityTotalThermoImage_6","LightBarrierActiveTaskDuration1_6","NumberEmptyFeeder_0",
                                     "NumberFuseDetected_6","NumberFuseEstimated_6","Pressure_6","ProcessCpuLoadNormalized_3",
                                     "ProcessMemoryConsumption_3","SharpnessImage_6","SmartMotorPositionError_6",
                                     "SmartMotorSpeed_6","Temperature_0","TemperatureThermoCam_6","TotalCpuLoadNormalized_3",
                                     "TotalMemoryConsumption_3","Vacuum_6","VacuumFusePicked_6","VacuumValveClosed_6"])
    return df

In [4]:
def Data_adjust(csv_name):
    list_df = [] 
    df_new = pd.DataFrame()
    source = pd.read_csv(csv_name)
    for i in range(0,len(source)):
        df = []
        row = source.iloc[i,1]
        row = row.strip("[]")
        texts = row.split("], [")
        for j in range(0,len(texts)):
            df.append(texts[j].split(', '))
        df = pd.DataFrame(df)
        name = source.iloc[i,0]
        for k in range(0,df.shape[1]):
            try:
                df.rename(columns={k:str(name)+'_'+str(k)},inplace=True)
            except:
                print("wrong code")
        list_df.append(df)
    df_new = pd.concat(list_df,axis = 1)
    return df_new

In [18]:
def Test_feature(df,time_steps):
    X = pd.DataFrame()
    test_feature_dict = {}
    df = df.fillna(0)  
    if len(df.columns) != 247:
        df["NumberFuseDetected_6"] = 0   
    df = Select_feature(df)  ## Select_feature to 36
    for i in range(36):
        feature = df.iloc[:,i]
        for j in range(feature.shape[0] - time_steps):
            time_step = np.array(df.iloc[j:j+time_steps,i])
            time_df = pd.DataFrame(time_step)
            time_df = time_df.T
            X = pd.concat([X,time_df],ignore_index = True)
        test_feature_dict[df.columns[i]] = X
        X = pd.DataFrame()
        test_feature_dict[df.columns[i]]=test_feature_dict[df.columns[i]].astype("float64")
    return test_feature_dict

In [50]:
 def Test_detect(feature_name,test_feature_dict):
    p=0;f=0;sensor_list=[];fail_dict={}
    for i in feature_name:
        with open(i+'.pickle.dat', 'rb') as handle:
            model = pickle.load(handle) #input feature's model
        y_pred=model.predict(test_feature_dict[i]) 
        if (Signal_detect(y_pred,i)=="Fail"): 
            f+=1
            sensor_list.append(i)
            df=pd.DataFrame()
            fail_df=test_feature_dict[i]
            for key in range(0,len(y_pred)):
                if y_pred[key] == 1 :
                    fail_sample=pd.DataFrame(fail_df.iloc[key,:])
                    fail_sample=fail_sample.T
                    df=pd.concat([df,fail_sample],ignore_index = True)
            df=df.fillna(0)
            fail_dict[i]=df #input bad sensor's sample
        else:
            p+=1
    if p/(p+f)>0.75:
        return "None","None"
    else:
        return fail_dict,sensor_list 

In [30]:
def Signal_detect(y_pred,i): #Sensor's pass or Fail
    a=0;b=0
    for z in y_pred:
        if z == 0:
            a+=1
        elif z == 1:
            b+=1
    if a/len(y_pred)>0.75:
        return("Pass")
    else:
        return("Fail")

In [23]:
def Label_predict(feature,x_test):
    sensor_label = []
    label={}
    label['2']=0;label['3']=0;label['4']=0;label['5']=0;label['7']=0;label['9']=0;label['11']=0;label['12']=0
    model = keras.models.load_model(feature+'_2')
    y_pred = model.predict_classes(x_test)
    for j in y_pred:
        if j == 0:
            label['2']+=1
        elif j == 1:
            label['3']+=1
        elif j == 2:
            label['4']+=1
        elif j == 3:
            label['5']+=1 
        elif j == 4:
            label['7']+=1
        elif j == 5:
            label['9']+=1
        elif j == 6:
            label['11']+=1
        elif j == 7:
            label['12']+=1
    rank=sorted(label.items(), key =lambda x : x[1],reverse=True) 
    return rank[0][0]  ## return the wrong sensor's label (2,3,4,5,7,9,11,12)

In [46]:
def Label_classfication(fail_dict):
    sensor_label=[]
    for key in fail_dict:    ##key == feature
        df = fail_dict[key]  
        row_list = []
        for i in range(df.shape[0]):
            row =pd.DataFrame(df.iloc[i,:])            
            row = np.array(row)
            row_list.append(row)
        x_test = np.asarray(row_list)
        x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
        rank = Label_predict(key,x_test)
        sensor_label.append(rank)
    maxlabel = max(sensor_label,key=sensor_label.count)
    return maxlabel

In [ ]:
#The Record Performance function store the information of the performance achieved in the 1st run of the classification
def RecordPerformance(Experiment, Label, Time, Ranking):

    if not os.path.exists('First'):
        os.makedirs('First')
    
    PerformanceOutput = open("First/%s.csv"%Experiment,"w")
    PerformanceOutput.write("Experiment;Label;Time;Ranking\n")
    PerformanceOutput.write(Experiment+";"+str(Label)+";"+str(Time)+";"+str(Ranking)+"\n")
    PerformanceOutput.close()
    
    return

#The CutExperiment function is used to cut the input experiment in the time order. For each experiment, the cut is performed accoring to the time to classification declared by the team 
def CutExperiment(FolderName, Experiment, Time):

    if not os.path.exists('Cut'):
        os.makedirs('Cut')
    
    data = pd.read_csv(FolderName+"/%s.csv"%Experiment,sep=",")  

    df = pd.DataFrame(columns = ["c1","c2"])
    
    for i in range(len(data)):
        field = data.iloc[i][0]
        records = eval(data.iloc[i][1])[:Time]

        df = df.append({"c1": field, "c2": records}, ignore_index=True)    
    
    columns =list(data.columns)
    df.columns = ["",columns[1]]
    df.to_csv("Cut/%s.csv"%Experiment,index=False)
    return

#ComparePerfomance check if the team achieved the same performance in the 1st and in the 2nd run 
def ComparePerformance(Experiment,Label, Time, Ranking):
    
    Performance = pd.read_csv("First/%s.csv"%Experiment,sep=";")        
        
    if(Performance["Label"].iloc[0]!=Label): return False
    if(Performance["Time"].iloc[0]!=Time): return False
    if(Performance["Ranking"].iloc[0]!=str(Ranking)): return False
        
    return True

#GetWorst function returns the worst performance in case the 1st and 2nd run performance does not match
def GetWorst(Experiment, Label, Time, Ranking):
    
    Performance = pd.read_csv("First/%s.csv"%Experiment,sep=";")  
    
    if(Performance["Label"].iloc[0]!=Label): return Performance["Label"].iloc[0], -1, Performance["Ranking"].iloc[0]
    if(Performance["Time"].iloc[0]!=Time): return Performance["Label"].iloc[0], Performance["Time"].iloc[0], Performance["Ranking"].iloc[0]
    if(Performance["Ranking"].iloc[0]!=str(Ranking)): return Performance["Label"].iloc[0], Performance["Time"].iloc[0], Performance["Ranking"].iloc[0]
    
    return

#Logperformance function stores the final performance. Only this performance will be used to compute the Penalty score of each team
def LogPerformance(Experiment,Label, Time, Ranking):

    if not os.path.exists('Results'):
        os.makedirs('Results')
        
    PerformanceOutput = open("Results/%s.csv"%Experiment,"w")
    PerformanceOutput.write("Experiment;Label;Time;Ranking\n")
    PerformanceOutput.write(Experiment+";"+str(Label)+";"+str(Time)+";"+str(Ranking)+"\n")
    PerformanceOutput.close()
    
    return

In [51]:
def main():
    
    FolderName = r"C:\Users\user\PHM\dataset"
    Experiment = "class_ 4_2_data.csv"
    Label, Time, Ranking = TestClassification(FolderName,Experiment)
    
    RecordPerformance(Experiment, Label, Time, Ranking)
    CutExperiment(FolderName,Experiment,Time)
    
    FolderName = "Cut/"
    Label, Time, Ranking = TestClassification(FolderName,Experiment)
    
    Equal = ComparePerformance(Experiment,Label, Time, Ranking)
    if(Equal==False):
        Label, Time, Ranking = GetWorst(Experiment,Label, Time, Ranking)
    
    LogPerformance(Experiment,Label, Time, Ranking)
    return

main()

D:\anaconda\envs\ML_class\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


            0          1          2          3          4          5   \
0    55.093750  56.075000  54.475000  56.800000  56.204545  57.375000   
1    56.075000  54.475000  56.800000  56.204545  57.375000  54.666667   
2    56.204545  57.375000  54.666667  55.727273  57.075000  56.350000   
3    57.375000  54.666667  55.727273  57.075000  56.350000  55.625000   
4    54.666667  55.727273  57.075000  56.350000  55.625000  55.175000   
..         ...        ...        ...        ...        ...        ...   
870  59.875000  59.000000  60.150000  58.925000  60.850000  59.750000   
871  59.000000  60.150000  58.925000  60.850000  59.750000  60.300000   
872  60.850000  59.750000  60.300000  60.700000  60.200000  59.300000   
873  60.700000  60.200000  59.300000  60.575000  59.525000  60.200000   
874  60.200000  59.300000  60.575000  59.525000  60.200000  60.150000   

            6          7       8       9   ...      60      61         62  \
0    54.666667  55.727273  57.075  56.350  ...

D:\anaconda\envs\ML_class\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


           0         1         2         3         4         5         6   \
0    0.000000  3.036333  3.201667  3.139000  2.989750  3.063667  3.233500   
1    3.123333  3.138250  3.063667  3.165000  4.096667  3.007750  2.846667   
2    3.165000  4.096667  3.007750  2.846667  3.011250  4.511000  3.038667   
3    3.033667  3.337333  3.182750  3.092000  2.929000  3.266667  3.548667   
4    3.337333  3.182750  3.092000  2.929000  3.266667  3.548667  3.187333   
..        ...       ...       ...       ...       ...       ...       ...   
487  3.087333  3.251750  3.194667  3.201000  3.363750  3.149667  3.122667   
488  3.153667  3.231000  3.630333  2.927333  2.954000  3.233333  3.111750   
489  2.927333  2.954000  3.233333  3.111750  3.203333  3.073667  3.164500   
490  3.164500  4.375333  3.140250  3.053000  3.265667  3.035000  3.190333   
491  3.140250  3.053000  3.265667  3.035000  3.190333  3.087250  3.233000   

           7         8         9   ...        60        61        62  \
0  

D:\anaconda\envs\ML_class\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


...

[[[55.09375   ]
  [56.075     ]
  [54.475     ]
  ...
  [57.3       ]
  [55.05      ]
  [56.375     ]]

 [[56.075     ]
  [54.475     ]
  [56.8       ]
  ...
  [55.05      ]
  [56.375     ]
  [57.275     ]]

 [[56.20454545]
  [57.375     ]
  [54.66666667]
  ...
  [55.925     ]
  [55.625     ]
  [55.75      ]]

 ...

 [[60.85      ]
  [59.75      ]
  [60.3       ]
  ...
  [60.775     ]
  [60.375     ]
  [59.8       ]]

 [[60.7       ]
  [60.2       ]
  [59.3       ]
  ...
  [60.675     ]
  [61.2       ]
  [59.7       ]]

 [[60.2       ]
  [59.3       ]
  [60.575     ]
  ...
  [61.2       ]
  [59.7       ]
  [59.65      ]]] (875, 70, 1)


D:\anaconda\envs\ML_class\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[[0.        ]
  [3.03633333]
  [3.20166667]
  ...
  [3.02533333]
  [3.23225   ]
  [3.129     ]]

 [[3.12333333]
  [3.13825   ]
  [3.06366667]
  ...
  [3.40166667]
  [3.225     ]
  [3.14333333]]

 [[3.165     ]
  [4.09666667]
  [3.00775   ]
  ...
  [3.58966667]
  [3.02375   ]
  [3.05033333]]

 ...

 [[2.92733333]
  [2.954     ]
  [3.23333333]
  ...
  [3.10333333]
  [3.07033333]
  [3.10125   ]]

 [[3.1645    ]
  [4.37533333]
  [3.14025   ]
  ...
  [3.126     ]
  [3.39525   ]
  [3.228     ]]

 [[3.14025   ]
  [3.053     ]
  [3.26566667]
  ...
  [3.228     ]
  [3.10633333]
  [3.03633333]]] (492, 70, 1)
[[[0.        ]
  [0.        ]
  [0.6605    ]
  ...
  [0.683     ]
  [0.69533333]
  [0.70833333]]

 [[0.        ]
  [0.6605    ]
  [0.69733333]
  ...
  [0.69533333]
  [0.70833333]
  [0.68466667]]

 [[0.6605    ]
  [0.69733333]
  [0.66733333]
  ...
  [0.70833333]
  [0.68466667]
  [0.727     ]]

 ...

 [[0.66833333]
  [0.66475   ]
  [0.65166667]
  ...
  [0.658     ]
  [0.702     ]
  [0.69725  

[[[15.5078125 ]
  [ 0.        ]
  [ 0.        ]
  ...
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]

 [[ 0.        ]
  [ 0.        ]
  [16.39453125]
  ...
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]

 [[21.5390625 ]
  [ 0.        ]
  [ 0.        ]
  ...
  [ 0.        ]
  [ 0.        ]
  [20.20898438]]

 ...

 [[ 0.        ]
  [ 0.        ]
  [ 0.        ]
  ...
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]

 [[ 0.        ]
  [18.1953125 ]
  [ 0.        ]
  ...
  [ 0.        ]
  [ 0.        ]
  [18.34765625]]

 [[20.578125  ]
  [ 0.        ]
  [ 0.        ]
  ...
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]] (269, 70, 1)
[[[-112.75]
  [-224.  ]
  [-448.  ]
  ...
  [-449.2 ]
  [-446.2 ]
  [-446.4 ]]

 [[-224.  ]
  [-448.  ]
  [-446.2 ]
  ...
  [-446.2 ]
  [-446.4 ]
  [-447.2 ]]

 [[-446.2 ]
  [-449.6 ]
  [-449.2 ]
  ...
  [-447.2 ]
  [-455.4 ]
  [-453.  ]]

 ...

 [[-446.8 ]
  [-445.25]
  [-444.6 ]
  ...
  [-447.2 ]
  [-446.  ]
  [-444.6 ]]

 [[-445.25]
  [-444.6 ]
  [-454

('4',
 161.55884838104248,
 ['Humidity',
  'TotalMemoryConsumption',
  'ProcessMemoryConsumption',
  'Pressure',
  'SmartMotorPositionError',
  'VacuumValveClosed',
  'NumberFuseEstimated',
  'Vacuum',
  'IntensityTotalImage',
  'NumberEmptyFeeder',
  'Temperature',
  'TemperatureThermoCamIntensityTotalThermoImage',
  'NumberFuseDetected',
  'FeederBackgroundIlluminationIntensity',
  'SharpnessImage',
  'FeederAction2',
  'DurationPickToPick',
  'FuseIntoFeeder',
  'TotalCpuLoadNormalized',
  'DurationRobotFromFeederToTestBench',
  'DurationRobotFromTestBenchToFeeder',
  'FusePicked',
  'FuseCycleDuration',
  'EPOSPosition',
  'ProcessCpuLoadNormalized',
  'FuseHeatSlope',
  'LightBarrierPassiveTaskDuration1',
  'FuseHeatSlopeNOK',
  'DurationTestBenchClosed',
  'VacuumFusePicked',
  'CpuTemperature',
  'FuseTestResult',
  'FeederAction4',
  'FuseHeatSlopeOK',
  'SmartMotorSpeed',
  'LightBarrierActiveTaskDuration1',
  'ValidFrameOptrisPIIRCamera',
  'EPOSVelocity',
  'ValidFrame',
  '